In [0]:
!pip install imutils

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imutils import paths

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.applications import VGG19
from keras.models import Model 
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
train_data_dir = "/content/drive/My Drive/train"
test_data_dir = "/content/drive/My Drive/test"
TRAIN = len(list(paths.list_images(train_data_dir)))
TEST = len(list(paths.list_images(test_data_dir)))
BS = 10
EPOCHS = 10
img_width, img_height = 300, 300

In [0]:
base_model = VGG19(weights = "imagenet", include_top=False, 
                   input_shape = (img_width, img_height, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = "relu")(x)
x = Dropout(0.4)(x)
x = Dense(256, activation = "relu")(x)
x = Dropout(0.1)(x)
preds = Dense(2, activation = "softmax")(x)

model = Model(input = base_model.input, output = preds)

for i,layer in enumerate(model.layers):
    print(i,layer.name)

for layer in model.layers[:14]:
    layer.trainable=False
for layer in model.layers[14:]:
    layer.trainable=True
    
model.summary()

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_conv4
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_conv4
16 block4_pool
17 block5_conv1
18 block5_conv2
19 block5_conv3
20 block5_conv4
21 block5_pool
22 flatten_1
23 dense_1
24 dropout_1
25 dense_2
26 dropout_2
27 dense_3
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


In [0]:
early = EarlyStopping(monitor = 'val_acc', min_delta = 0, 
                      patience = 10, verbose= 1 , mode = 'auto')

model.compile(loss = "categorical_crossentropy", 
              optimizer = SGD(lr=0.001, momentum=0.9), 
              metrics=["accuracy"])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

trainAug = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               horizontal_flip = True,
                               fill_mode = "nearest")

In [0]:
testAug = ImageDataGenerator(rescale = 1./255,
                             fill_mode = "nearest")

In [0]:
trainGen = trainAug.flow_from_directory('/content/drive/My Drive/train',
                                         target_size = (img_width, img_height),
                                         batch_size = BS,
                                         shuffle = True,
                                         class_mode = 'categorical')

Found 1400 images belonging to 2 classes.


In [0]:
testGen = testAug.flow_from_directory('/content/drive/My Drive/test',
                                    target_size = (img_width, img_height),
                                    batch_size = BS,
                                    class_mode = 'categorical')

Found 349 images belonging to 2 classes.


In [0]:
classes = trainGen.class_indices    
print(classes)
H = model.fit_generator(trainGen, epochs = EPOCHS,
                        steps_per_epoch = TRAIN // BS)
print("CNN Trained")

{'neg': 0, 'pos': 1}
Epoch 1/10
140/140 [==============================] - 2554s 18s/step - loss: 0.5802 - accuracy: 0.7343
Epoch 2/10
140/140 [==============================] - 2563s 18s/step - loss: 0.3520 - accuracy: 0.8443
Epoch 3/10
140/140 [==============================] - 2682s 19s/step - loss: 0.2336 - accuracy: 0.9121
Epoch 4/10
140/140 [==============================] - 2704s 19s/step - loss: 0.2040 - accuracy: 0.9186
Epoch 5/10
140/140 [==============================] - 2670s 19s/step - loss: 0.2004 - accuracy: 0.9214
Epoch 6/10
140/140 [==============================] - 2620s 19s/step - loss: 0.1237 - accuracy: 0.9593
Epoch 7/10
140/140 [==============================] - 2610s 19s/step - loss: 0.1175 - accuracy: 0.9557
Epoch 8/10
140/140 [==============================] - 2662s 19s/step - loss: 0.0762 - accuracy: 0.9736
Epoch 9/10
140/140 [==============================] - 2554s 18s/step - loss: 0.0842 - accuracy: 0.9757
Epoch 10/10
140/140 [==============================]

In [0]:
model.save('/content/drive/My Drive/model.h5')
print("CNN Saved")

CNN Saved
